### PyTorchを使ってニューラルネットワークを構築する

In [ ]:
'''
1. データの読み込みと前処理
'''
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# ダウンロード先のディレクトリ
root = './data'

# トランスフォーマーオブジェクトを生成
transform = transforms.Compose([
    transforms.ToTensor(), # Tensorオブジェクトに変換
    transforms.Normalize((0.5), (0.5)), # 平均0.5、標準偏差0.5に正規化
    lambda x: x.view(-1) # データの形状を(28, 28)から(784,)に変換
])

# 訓練用データの読み込み（60000セット）
f_mnist_train = datasets.FashionMNIST(
    root=root,
    download=True,
    train=True,
    transform=transform
)
# テスト用データの読み込み（10000セット）
f_mnist_test = datasets.FashionMNIST(
    root=root,
    download=True,
    train=False,
    transform=transform
)

# ミニバッチのサイズ
batch_size = 64
# 訓練用のデータローダー
train_dataloader = DataLoader(
    f_mnist_train,
    batch_size= batch_size,
    shuffle=True
)
# テスト用のデータローダー
test_dataloader = DataLoader(
    f_mnist_test,
    batch_size= batch_size,
    shuffle=False
)

# データローダーが返すミニバッチの先頭データの形状を出力
for (x, t) in train_dataloader:
    print(x.shape)
    print(t.shape)
    break

for (x, t) in test_dataloader:
    print(x.shape)
    print(t.shape)
    break

In [5]:
'''
2. モデルの定義
'''
import torch
import torch.nn as nn

class MLP(nn.Module):
    '''
    多層パーセプトロン
    Attributes:
        l1(Linear): 隠れ層
        l2(Linear): 出力層
        d1(Dropout): ドロップアウト
    '''
    def __init__(self, input_dim, hidden_dim, output_dim):
        '''
        モデルの初期化を行う
        Parameters:
            input_dim(int): 入力する1データあたりの値の形状
            hidden_dim(int): 隠れ層のユニット数
            output_dim(int): 出力層のユニット数
        '''
        # スーパークラスの__init__()を実行
        super().__init__()
        # 隠れ層
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # ドロップアウト
        self.d1 = nn.Dropout(0.5)
        # 出力層
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        '''
        MLPの順伝播処理を行う
        Parameters:
            x(ndarray(float32)): 訓練データまたはテストデータ
        Returns(float32):
            出力層からの出力値
        '''
        # レイヤー、活性化関数に前ユニットからの出力を入力する
        x = self.fc1(x)
        x = torch.sigmoid(x)
        x = self.d1(x)
        x = self.fc2(x)
        return x

In [6]:
'''
3. モデルの生成
'''
# 使用可能なデバイスを取得する
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルオブジェクトを生成し、使用可能なデバイスを設定する
model = MLP(784, 256, 10).to(device)

model

MLP(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (d1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)

In [7]:
'''
4. 損失関数とオプティマイザーの生成
'''
import torch.optim

# クロスエントロピー誤差のオブジェクトを生成
criterion = nn.CrossEntropyLoss()
# 勾配降下アルゴリズムを使用するオプティマイザーを生成
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [8]:
'''
5. train_step()関数の定義
'''
def train_step(x, t):
    '''
    バックプロパゲーションによるパラメータ更新を行う
    Parameters:
        x: 訓練データ
        t: 正解ラベル
    Returns:
        MLPの出力と正解ラベルのクロスエントロピー誤差
    '''
    model.train()
    preds = model(x)
    loss = criterion(preds, t)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, preds

In [9]:
'''
6. test_step()関数の定義
'''
def test_step(x, t):
    '''
    テストデータを入力して損失と予測値を返す
    Paramters:
        x: テストデータ
        t: 正解ラベル
    Returns:
        MLPの出力と正解ラベルのクロスエントロピー誤差
    '''
    model.eval()
    preds = model(x)
    loss = criterion(preds, t)

    return loss, preds

In [10]:
'''
7. 学習の進捗を監視し早期終了判定を行うクラス
'''
class EarlyStopping:
    def __init__(self, patience=1.0, verbose=0):
        '''
        Parameters:
            patience(int): 監視するエポック数
            verbose(int): 早期終了メッセージの出力フラグ
        '''
        # インスタンス変数の初期化
        # 監視中のエポック数のカウンターを初期化
        self.epoch = 0
        # 比較対象の損失を無限大'inf''で初期化
        self.pre_loss = float('inf')
        # 監視対象のエポック数をパラメータで初期化
        self.patience = patience
        # 早期終了メッセージの出力フラグをパラメータで初期化
        self.verbose = verbose
    
    def __call__(self, current_loss):
        '''
        Parameters:
            current_loss(float): 1エポック終了後の検証データの損失
        Returns:
            True: 監視回数の上限までに前エポックの損失を超えた場合
            False: 監視回数の上限までに前エポックの損失を超えない場合
        '''
        # 前エポックの損失より大きくなった場合
        if self.pre_loss < current_loss:
            self.epoch += 1
            # 監視回数の上限に達した場合
            if self.epoch > self.patience:
                if self.verbose:
                    print('early stoopping')
                return True
        # 前エポックの損失以下の場合
        else:
            self.epoch = 0
            self.pre_loss = current_loss
        
        return False

In [ ]:
%%time
'''
8. モデルを使用して学習する
'''
from sklearn.metrics import accuracy_score

# エポック数
epochs = 200
# 損失と精度の履歴を保存するためにdictオブジェクト
history = {
    'loss': [],
    'accuracy': [],
    'test_loss': [],
    'test_accuracy': [],
}
# 早期終了の判定を行うオブジェクトを生成
ers = EarlyStopping(patience=5, verbose=1)
# 学習を行う
for epoch in range(epochs):
    train_loss = 0.
    train_acc = 0.
    test_loss = 0.
    test_acc = 0.

    # 1ステップにおける訓練用ミニバッチを使用した学習
    for (x, t) in train_dataloader:
        # torch.Tensorオブジェクトにデバイス割り当てる
        x, t = x.to(device), t.to(device)
        loss, preds = train_step(x, t)
        train_loss += loss.item()
        train_acc += accuracy_score(
            t.tolist(),
            preds.argmax(dim=-1).tolist()
        )

    # 1ステップにおけるテストミニバッチを使用した学習
    for (x, t) in test_dataloader:
        # torch.Tensorオブジェクトにデバイス割り当てる
        x, t = x.to(device), t.to(device)
        loss, preds = test_step(x, t)
        train_loss += loss.item()
        train_acc += accuracy_score(
            t.tolist(),
            preds.argmax(dim=-1).tolist()
        )

    avg_train_loss = train_loss / len(train_dataloader)
    avg_train_acc = train_acc / len(train_dataloader)
    avg_test_loss = test_loss / len(test_dataloader)
    avg_test_acc = test_acc / len(test_dataloader)

    # 訓練データの履歴を保存する
    history['loss'].append(avg_train_loss)
    history['accuracy'].append(avg_train_acc)
    # テストデータの履歴を保存する
    history['test_loss'].append(avg_test_loss)
    history['test_accuracy'].append(avg_test_acc)

    # 1エポックごとに結果を出力
    if (epoch + 1) % 1 == 0:
        print(
            'epoch({}) train_loss: {:.4} train_acc: {:.4} val_loss: {:.4} val_loss: {:.4}'
            .format(
                epoch + 1,
                avg_train_loss,
                avg_train_acc,
                avg_test_loss,
                avg_test_acc,
            )
        )
    # テストデータの損失をEarlyStoppingオブジェクトに渡して早期終了を判定
    if ers(avg_test_loss):
        break

In [ ]:
'''
9. 損失と精度の推移をグラフにする
'''
import matplotlib.pyplot as plt
%matplotlib inline

# 損失
plt.plot(
    history['loss'],
    marker='.',
    label='loss (Training)'
)
plt.plot(
    history['test_loss'],
    marker='.',
    label='loss (Test)'
)
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

# 精度
plt.plot(
    history['accuracy'],
    marker='.',
    label='accuracy (Training)'
)
plt.plot(
    history['test_accuracy'],
    marker='.',
    label='accuracy (Test)'
)
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()